# 2.2. Complex Machine Learning Models and Keras Part 1

## The following script contains the following:
### 1. Import data libraries and data sets
### 2. Data preprocessing
### 3. Build and run CNN keras models
### 4. Run confusion matrix

## 01. Import data libraries and data sets

In [2]:
# Import libraries.

import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Activation, Dropout, Reshape, Permute
from keras.utils import to_categorical

In [3]:
# Supress scientific notation for easier analysis profiling
pd.set_option('display.float_format', '{:.2f}'.format)

# Set display options to show all columns without truncation
pd.set_option('display.max_columns', None)

In [4]:
# Create path
path = r'C:\Users\Quinn\Documents\CF - Data Analysis\Machine Learning\ClimateWins'

In [5]:
# Import the dataset
climate = pd.read_csv(os.path.join(path, '02 Data Sets', 'Dataset-weather-prediction-dataset-processed.csv'))
climate.head(5)

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_wind_speed,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_wind_speed,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_snow_depth,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,GDANSK_cloud_cover,GDANSK_humidity,GDANSK_precipitation,GDANSK_snow_depth,GDANSK_temp_mean,GDANSK_temp_min,GDANSK_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_snow_depth,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_wind_speed,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_wind_speed,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_wind_speed,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_wind_speed,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_snow_depth,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_wind_speed,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_snow_depth,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,ROMA_cloud_cover,ROMA_wind_speed,ROMA_humidity,ROMA_pressure,ROMA_sunshine,ROMA_temp_mean,SONNBLICK_cloud_cover,SONNBLICK_wind_speed,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,TOURS_wind_speed,TOURS_humidity,TOURS_pressure,TOURS_global_radiation,TOURS_precipitation,TOURS_temp_mean,TOURS_temp_min,TOURS_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.10,0.85,1.02,0.32,0.09,0,0.70,6.50,0.80,10.90,1,0.81,1.02,0.88,0.00,7.00,3.70,-0.90,7.90,4,0.67,1.02,0.44,0.01,2.30,2.40,-0.40,5.10,7,7.70,0.85,1.00,0.07,0.25,0.00,9.30,7.40,11.00,8,5.40,0.83,1.02,0.12,0.08,0,0.00,10.00,7.00,11.50,8,0.91,0.00,0,0.80,-0.30,1.60,7,0.91,1.00,0.13,0.22,0,0.00,10.60,9.40,8.30,2.90,0.82,1.01,0.28,0.48,1.60,7.90,3.90,9.40,8,1.40,1.00,1.02,0.20,0.00,0.00,-0.60,-1.90,0.50,7,8.70,0.83,1.01,0.22,0.32,1.00,9.50,8.50,11.10,6,0.00,0.92,1.03,0.53,0.00,1.40,7.60,4.40,10.80,5,0.67,0.20,0.10,0,0.00,6.90,1.10,10.40,8,4.00,0.98,1.00,0.04,1.14,0,0.00,4.90,3.80,5.90,3,2.60,0.73,1.02,7.10,7.80,4,4.50,0.73,1.03,0.48,0.01,2.30,-5.90,-8.50,-3.20,5,1.01,0.05,0.32,0.00,4.20,2.20,4.90,3.80,0.76,1.02,1.54,0.44,10.00,7.80,12.20,5,0.88,1.00,0.45,0

In [6]:
pleasant = pd.read_csv(os.path.join(path, '02 Data Sets', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
pleasant.head(5)

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### I would choose CNN model over RNN keras model even though we wont be looking at any data with images, the CNN model works better than RNN because it also has advantages with analyzing numerical data, while the RNN model has advantages in text, writing, and speech.

## 02. Preprocessing data frames

In [7]:
# Check for all columsn
print(climate.columns.tolist())

['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_wind_speed', 'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_snow_depth', 'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max', 'BELGRADE_cloud_cover', 'BELGRADE_humidity', 'BELGRADE_pressure', 'BELGRADE_global_radiation', 'BELGRADE_precipitation', 'BELGRADE_sunshine', 'BELGRADE_temp_mean', 'BELGRADE_temp_min', 'BELGRADE_temp_max', 'BUDAPEST_cloud_cover', 'BUDAPEST_humidity', 'BUDAPEST_pressure', 'BUDAPEST_global_radiation', 'BUDAPEST_precipitation', 'BUDAPEST_sunshine', 'BUDAPEST_temp_mean', 'BUDAPEST_temp_min', 'BUDAPEST_temp_max', 'DEBILT_cloud_cover', 'DEBILT_wind_speed', 'DEBILT_humidity', 'DEBILT_pressure', 'DEBILT_global_radiation', 'DEBILT_precipitation', 'DEBILT_sunshine', 'DEBILT_temp_mean', 'DEBILT_temp_min', 'DEBILT_temp_max', 'DUSSELDORF_cloud_cover', 'DUSSELDORF_wind_speed', 'DUSSELDORF_humidity', 'DUSSELDORF_pressure', 'DUSSELDORF_global_radiation', 'DUSSELDORF_pr

####  After looking closely at the columns list, there are 2 observations with the missing years and inconsistencies: wind_speed and snow_depth.
#### We also do not need observations from 3 station: GDANSK, ROMA, TOURS

In [8]:
# Drop 3 weather stations
climate = climate.drop([
                         # GDANSK features
                         'GDANSK_cloud_cover', 'GDANSK_humidity',
                         'GDANSK_precipitation', 'GDANSK_snow_depth',
                         'GDANSK_temp_mean', 'GDANSK_temp_min', 'GDANSK_temp_max',
                         # ROMA features
                         'ROMA_cloud_cover', 'ROMA_humidity', 'ROMA_pressure',
                         'ROMA_sunshine', 'ROMA_temp_mean', 'ROMA_wind_speed',
                         # TOURS features
                         'TOURS_humidity', 'TOURS_pressure', 'TOURS_wind_speed',
                         'TOURS_global_radiation', 'TOURS_precipitation',
                         'TOURS_temp_mean', 'TOURS_temp_min', 'TOURS_temp_max'], axis=1)
climate

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_wind_speed,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_wind_speed,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_snow_depth,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_snow_depth,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_wind_speed,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_wind_speed,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_wind_speed,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_wind_speed,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_snow_depth,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_wind_speed,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_snow_depth,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_wind_speed,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.10,0.85,1.02,0.32,0.09,0,0.70,6.50,0.80,10.90,1,0.81,1.02,0.88,0.00,7.00,3.70,-0.90,7.90,4,0.67,1.02,0.44,0.01,2.30,2.40,-0.40,5.10,7,7.70,0.85,1.00,0.07,0.25,0.00,9.30,7.40,11.00,8,5.40,0.83,1.02,0.12,0.08,0,0.00,10.00,7.00,11.50,7,0.91,1.00,0.13,0.22,0,0.00,10.60,9.40,8.30,2.90,0.82,1.01,0.28,0.48,1.60,7.90,3.90,9.40,8,1.40,1.00,1.02,0.20,0.00,0.00,-0.60,-1.90,0.50,7,8.70,0.83,1.01,0.22,0.32,1.00,9.50,8.50,11.10,6,0.00,0.92,1.03,0.53,0.00,1.40,7.60,4.40,10.80,5,0.67,0.20,0.10,0,0.00,6.90,1.10,10.40,8,4.00,0.98,1.00,0.04,1.14,0,0.00,4.90,3.80,5.90,4,4.50,0.73,1.03,0.48,0.01,2.30,-5.90,-8.50,-3.20,5,1.01,0.05,0.32,0.00,4.20,2.20,4.90,5,0.88,1.00,0.45,0.34,0,4.70,8.50,6.00,10.90
1,19600102,1,6,2.10,0.84,1.02,0.36,1.05,0,1.10,6.10,3.30,10.10,6,0.84,1.02,0.25,0.00,0.00,2.90,2.20,4.40,4,0.67,1.02,0.18,0.31,0.00,2.30,1.40,3.10,8,4.10,0.90,1.01,0.14,0.06,0.10,7.70,6.40,8.30,8,3.60,0.89,1.02,0.18,0.66,0,0.50,8.20,7.40,11.00,7,0.98,1.01,0.13,0.23,0,0.00,6.10,3.90,10.60,1.90,0.86,1.01,0.12,0.27,0.00,7.70,6.80,9.10,6,1.40,0.94,1.02,0.56,0.13,3.20,2.10,-1.30,5.50,8,5.70,0.92,1.01,0.17,1.34,0.40,8.60,7.

In [9]:
#Drop columns ending with '_snow_depth'
climate = climate.drop(columns=climate.filter(like='_snow_depth').columns)
climate

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_wind_speed,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_wind_speed,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_wind_speed,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_wind_speed,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_wind_speed,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_wind_speed,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_wind_speed,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_wind_speed,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.10,0.85,1.02,0.32,0.09,0.70,6.50,0.80,10.90,1,0.81,1.02,0.88,0.00,7.00,3.70,-0.90,7.90,4,0.67,1.02,0.44,0.01,2.30,2.40,-0.40,5.10,7,7.70,0.85,1.00,0.07,0.25,0.00,9.30,7.40,11.00,8,5.40,0.83,1.02,0.12,0.08,0.00,10.00,7.00,11.50,7,0.91,1.00,0.13,0.22,0.00,10.60,9.40,8.30,2.90,0.82,1.01,0.28,0.48,1.60,7.90,3.90,9.40,8,1.40,1.00,1.02,0.20,0.00,0.00,-0.60,-1.90,0.50,7,8.70,0.83,1.01,0.22,0.32,1.00,9.50,8.50,11.10,6,0.00,0.92,1.03,0.53,0.00,1.40,7.60,4.40,10.80,5,0.67,0.20,0.10,0.00,6.90,1.10,10.40,8,4.00,0.98,1.00,0.04,1.14,0.00,4.90,3.80,5.90,4,4.50,0.73,1.03,0.48,0.01,2.30,-5.90,-8.50,-3.20,5,1.01,0.05,0.32,0.00,4.20,2.20,4.90,5,0.88,1.00,0.45,0.34,4.70,8.50,6.00,10.90
1,19600102,1,6,2.10,0.84,1.02,0.36,1.05,1.10,6.10,3.30,10.10,6,0.84,1.02,0.25,0.00,0.00,2.90,2.20,4.40,4,0.67,1.02,0.18,0.31,0.00,2.30,1.40,3.10,8,4.10,0.90,1.01,0.14,0.06,0.10,7.70,6.40,8.30,8,3.60,0.89,1.02,0.18,0.66,0.50,8.20,7.40,11.00,7,0.98,1.01,0.13,0.23,0.00,6.10,3.90,10.60,1.90,0.86,1.01,0.12,0.27,0.00,7.70,6.80,9.10,6,1.40,0.94,1.02,0.56,0.13,3.20,2.10,-1.30,5.50,8,5.70,0.92,1.01,0.17,1.34,0.40,8.60,7.50,9.90,7,0.80,0.86,1.03,0.46,0.00,0.90,9.80,7.40,12.20,6,0.72,0.61,0.30,5.10,6.20,4.20,10.20,8,5.10,0.62,1.01,0.04,0.00,0.00,3.40,2.

In [10]:
#Drop columns ending with '_wind_speed'
climate = climate.drop(columns=climate.filter(like='_wind_speed').columns)
climate

,DATE,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,0.85,1.02,0.32,0.09,0.70,6.50,0.80,10.90,1,0.81,1.02,0.88,0.00,7.00,3.70,-0.90,7.90,4,0.67,1.02,0.44,0.01,2.30,2.40,-0.40,5.10,7,0.85,1.00,0.07,0.25,0.00,9.30,7.40,11.00,8,0.83,1.02,0.12,0.08,0.00,10.00,7.00,11.50,7,0.91,1.00,0.13,0.22,0.00,10.60,9.40,8.30,0.82,1.01,0.28,0.48,1.60,7.90,3.90,9.40,8,1.00,1.02,0.20,0.00,0.00,-0.60,-1.90,0.50,7,0.83,1.01,0.22,0.32,1.00,9.50,8.50,11.10,6,0.92,1.03,0.53,0.00,1.40,7.60,4.40,10.80,5,0.67,0.20,0.10,0.00,6.90,1.10,10.40,8,0.98,1.00,0.04,1.14,0.00,4.90,3.80,5.90,4,0.73,1.03,0.48,0.01,2.30,-5.90,-8.50,-3.20,5,1.01,0.05,0.32,0.00,4.20,2.20,4.90,5,0.88,1.00,0.45,0.34,4.70,8.50,6.00,10.90
1,19600102,1,6,0.84,1.02,0.36,1.05,1.10,6.10,3.30,10.10,6,0.84,1.02,0.25,0.00,0.00,2.90,2.20,4.40,4,0.67,1.02,0.18,0.31,0.00,2.30,1.40,3.10,8,0.90,1.01,0.14,0.06,0.10,7.70,6.40,8.30,8,0.89,1.02,0.18,0.66,0.50,8.20,7.40,11.00,7,0.98,1.01,0.13,0.23,0.00,6.10,3.90,10.60,0.86,1.01,0.12,0.27,0.00,7.70,6.80,9.10,6,0.94,1.02,0.56,0.13,3.20,2.10,-1.30,5.50,8,0.92,1.01,0.17,1.34,0.40,8.60,7.50,9.90,7,0.86,1.03,0.46,0.00,0.90,9.80,7.40,12.20,6,0.72,0.61,0.30,5.10,6.20,4.20,10.20,8,0.62,1.01,0.04,0.00,0.00,3.40,2.80,4.90,6,0.97,1.03,0.21,0.61,0.00,-9.50,-10.50,-8.50,5,1.01,0.05,0.06,0.00,4.00,3.00,5.00,7,0.91,1.00,0.25,0.84,0.70,8.90,5.60,12.10
2,19600103,1,8,0.90,1.02,0.18,0.30,0.00,8.50,5.10,9.90,6,0.77,1.02,0.67,0.00,3.50,3.10,-0.50,6.40,4,0.67,1.02,0.30,0.00,0.60

#### Immute data from nearest stations (Ljubljana is near Kassel, Sonnblick is near Munchen, and Olso is close enough to Stockholm

In [11]:
climate['KASSEL_cloud_cover'] = climate['LJUBLJANA_cloud_cover']
climate['MUNCHENB_pressure'] = climate['SONNBLICK_pressure']
climate['STOCKHOLM_humidity'] = climate['OSLO_humidity']
climate

,DATE,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,KASSEL_cloud_cover,MUNCHENB_pressure,STOCKHOLM_humidity
0,19600101,1,7,0.85,1.02,0.32,0.09,0.70,6.50,0.80,10.90,1,0.81,1.02,0.88,0.00,7.00,3.70,-0.90,7.90,4,0.67,1.02,0.44,0.01,2.30,2.40,-0.40,5.10,7,0.85,1.00,0.07,0.25,0.00,9.30,7.40,11.00,8,0.83,1.02,0.12,0.08,0.00,10.00,7.00,11.50,7,0.91,1.00,0.13,0.22,0.00,10.60,9.40,8.30,0.82,1.01,0.28,0.48,1.60,7.90,3.90,9.40,8,1.00,1.02,0.20,0.00,0.00,-0.60,-1.90,0.50,7,0.83,1.01,0.22,0.32,1.00,9.50,8.50,11.10,6,0.92,1.03,0.53,0.00,1.40,7.60,4.40,10.80,5,0.67,0.20,0.10,0.00,6.90,1.10,10.40,8,0.98,1.00,0.04,1.14,0.00,4.90,3.80,5.90,4,0.73,1.03,0.48,0.01,2.30,-5.90,-8.50,-3.20,5,1.01,0.05,0.32,0.00,4.20,2.20,4.90,5,0.88,1.00,0.45,0.34,4.70,8.50,6.00,10.90,8,1.03,0.98
1,19600102,1,6,0.84,1.02,0.36,1.05,1.10,6.10,3.30,10.10,6,0.84,1.02,0.25,0.00,0.00,2.90,2.20,4.40,4,0.67,1.02,0.18,0.31,0.00,2.30,1.40,3.10,8,0.90,1.01,0.14,0.06,0.10,7.70,6.40,8.30,8,0.89,1.02,0.18,0.66,0.50,8.20,7.40,11.00,7,0.98,1.01,0.13,0.23,0.00,6.10,3.90,10.60,0.86,1.01,0.12,0.27,0.00,7.70,6.80,9.10,6,0.94,1.02,0.56,0.13,3.20,2.10,-1.30,5.50,8,0.92,1.01,0.17,1.34,0.40,8.60,7.50,9.90,7,0.86,1.03,0.46,0.00,0.90,9.80,7.40,12.20,6,0.72,0.61,0.30,5.10,6.20,4.20,10.20,8,0.62,1.01,0.04,0.00,0.00,3.40,2.80,4.90,6,0.97,1.03,0.21,0.61,0.00,-9.50,-10.50,-8.50,5,1.01,0.05,0.06,0.00,4.00,3.00,5.00,7,0.91,1.00,0.25,0.84,0.70,8.90,5.60,12.10,6,1.03,0.62
2,19600103,1,8,0.90,1.02,0.18,0.30,0.00,8.50

In [12]:
# Drop the "DATE , MONTH" columsn
df = climate.drop(columns = ['DATE','MONTH'])
df

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,KASSEL_cloud_cover,MUNCHENB_pressure,STOCKHOLM_humidity
0,7,0.85,1.02,0.32,0.09,0.70,6.50,0.80,10.90,1,0.81,1.02,0.88,0.00,7.00,3.70,-0.90,7.90,4,0.67,1.02,0.44,0.01,2.30,2.40,-0.40,5.10,7,0.85,1.00,0.07,0.25,0.00,9.30,7.40,11.00,8,0.83,1.02,0.12,0.08,0.00,10.00,7.00,11.50,7,0.91,1.00,0.13,0.22,0.00,10.60,9.40,8.30,0.82,1.01,0.28,0.48,1.60,7.90,3.90,9.40,8,1.00,1.02,0.20,0.00,0.00,-0.60,-1.90,0.50,7,0.83,1.01,0.22,0.32,1.00,9.50,8.50,11.10,6,0.92,1.03,0.53,0.00,1.40,7.60,4.40,10.80,5,0.67,0.20,0.10,0.00,6.90,1.10,10.40,8,0.98,1.00,0.04,1.14,0.00,4.90,3.80,5.90,4,0.73,1.03,0.48,0.01,2.30,-5.90,-8.50,-3.20,5,1.01,0.05,0.32,0.00,4.20,2.20,4.90,5,0.88,1.00,0.45,0.34,4.70,8.50,6.00,10.90,8,1.03,0.98
1,6,0.84,1.02,0.36,1.05,1.10,6.10,3.30,10.10,6,0.84,1.02,0.25,0.00,0.00,2.90,2.20,4.40,4,0.67,1.02,0.18,0.31,0.00,2.30,1.40,3.10,8,0.90,1.01,0.14,0.06,0.10,7.70,6.40,8.30,8,0.89,1.02,0.18,0.66,0.50,8.20,7.40,11.00,7,0.98,1.01,0.13,0.23,0.00,6.10,3.90,10.60,0.86,1.01,0.12,0.27,0.00,7.70,6.80,9.10,6,0.94,1.02,0.56,0.13,3.20,2.10,-1.30,5.50,8,0.92,1.01,0.17,1.34,0.40,8.60,7.50,9.90,7,0.86,1.03,0.46,0.00,0.90,9.80,7.40,12.20,6,0.72,0.61,0.30,5.10,6.20,4.20,10.20,8,0.62,1.01,0.04,0.00,0.00,3.40,2.80,4.90,6,0.97,1.03,0.21,0.61,0.00,-9.50,-10.50,-8.50,5,1.01,0.05,0.06,0.00,4.00,3.00,5.00,7,0.91,1.00,0.25,0.84,0.70,8.90,5.60,12.10,6,1.03,0.62
2,8,0.90,1.02,0.18,0.30,0.00,8.50,5.10,9.90,6,0.77,1.02,0.67,0.00,3.50,3.10,-

In [13]:
# Drop the "DATE" from pleasent days dataset
pleasant = pleasant.drop(columns = ['DATE'])
pleasant

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22946,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22947,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22948,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
# Convert df to array.
X = df.to_numpy()

In [15]:
# Reshape df
X = df.values.reshape(-1, 15, 9)

# Ensure the labels are in the same shape as X
y = pleasant.values.reshape(-1, 15)

def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [16]:
X.shape

(22950, 15, 9)

In [17]:
y.shape

(22950, 15)

In [18]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(16065, 15, 9) (16065, 15)
(6885, 15, 9) (6885, 15)


## 03. Create a Keras layered model

In [20]:
X_train

array([[[ 2.0000e+00,  6.5000e-01,  1.0180e+00, ...,  2.0500e+01,
          1.1900e+01,  2.7700e+01],
        [ 3.0000e+00,  5.3000e-01,  1.0201e+00, ...,  2.3800e+01,
          1.6300e+01,  2.8100e+01],
        [ 4.0000e+00,  6.7000e-01,  1.0170e+00, ...,  2.1600e+01,
          1.4500e+01,  2.9000e+01],
        ...,
        [ 1.0307e+00,  2.7800e+00,  0.0000e+00, ...,  9.9000e+00,
          5.0000e+00,  1.0156e+00],
        [ 2.0500e+00,  2.0000e-02,  9.1000e+00, ...,  5.0000e+00,
          8.2000e-01,  1.0142e+00],
        [ 1.7400e+00,  4.1000e-01,  3.4000e+00, ...,  0.0000e+00,
          1.0307e+00,  9.3000e-01]],

       [[ 1.0000e+00,  4.8000e-01,  1.0180e+00, ...,  1.1700e+01,
          2.5000e+00,  1.9300e+01],
        [ 8.0000e+00,  8.0000e-01,  1.0187e+00, ...,  1.0500e+01,
          8.3000e+00,  1.4200e+01],
        [ 8.0000e+00,  6.4000e-01,  1.0211e+00, ...,  9.8000e+00,
          4.5000e+00,  1.5400e+01],
        ...,
        [ 1.0322e+00,  3.4100e+00,  0.0000e+00, ..., -

In [21]:
# Print number of cities in X_train data set.

len(X_train[0])

15

In [22]:
# Print number of weather observations in X_train data set.

len(X_train[0][0])

9

In [23]:
len(y_train[0])

15

## CNN Model 1

In [24]:
# Adjust model hyperparameters
epochs = 32
batch_size = 8
n_hidden = 16
kernel_size = 2 # Adjust the kernel size here

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

# Implement complex layers
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=kernel_size, activation='relu', input_shape=(timesteps, input_dim)))  # Use the adjusted kernel size
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #relu #sigmoid #tanh #softmax

# Build model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Run model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/32


C:\Users\Quinn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2009/2009 ━━━━━━━━━━━━━━━━━━━━ 3s 899us/step - accuracy: 0.1011 - loss: 3788.1885
Epoch 2/32
2009/2009 ━━━━━━━━━━━━━━━━━━━━ 2s 890us/step - accuracy: 0.1290 - loss: 81191.3203
Epoch 3/32
2009/2009 ━━━━━━━━━━━━━━━━━━━━ 2s 896us/step - accuracy: 0.1208 - loss: 313102.6875
Epoch 4/32
2009/2009 ━━━━━━━━━━━━━━━━━━━━ 2s 913us/step - accuracy: 0.1328 - loss: 721574.4375
Epoch 5/32
2009/2009 ━━━━━━━━━━━━━━━━━━━━ 2s 998us/step - accuracy: 0.1247 - loss: 1353620.3750
Epoch 6/32
2009/2009 ━━━━━━━━━━━━━━━━━━━━ 2s 896us/step - accuracy: 0.1279 - loss: 2203530.5000
Epoch 7/32
2009/2009 ━━━━━━━━━━━━━━━━━━━━ 2s 899us/step - accuracy: 0.1297 - loss: 3367675.5000
Epoch 8/32
2009/2009 ━━━━━━━━━━━━━━━━━━━━ 2s 902us/step - accuracy: 0.1286 - loss: 4950267.0000
Epoch 9/32
2009/2009 ━━━━━━━━━━━━━━━━━━━━ 2s 902us/step - accuracy: 0.1294 - loss: 6645931.0000
Epoch 10/32
2009/2009 ━━━━━━━━━━━━━━━━━━━━ 2s 900us/step - accuracy: 0.1274 - loss: 9242639.0000
Epoch 11/32
2009/2009 ━━━━━━━━━━━━━━━━━━━━ 2s 911us/step 

#### Notes: relatively low accuracy and exceptionally high loss, the model may be too simple to capture complex data patterns

### Confusion matrix

In [25]:
# Define cities.
cities = [
    "Basel",
    "Belgrade",
    "Budapest",
    "Debilt",
    "Dusseldorf",
    "Heathrow",
    "Kassel",
    "Ljubljana",
    "Maastricht",
    "Madrid",
    "Munchen",
    "Oslo",
    "Sonnblick",
    "Stockholm",
    "Valentia"
]

In [26]:
# Define confusion matrix to see results.

def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([cities[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([cities[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [27]:
# Print results.

print(confusion_matrix(y_test, model.predict(X_test)))

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step
Pred        Basel  Belgrade  Debilt  Dusseldorf  Heathrow  Kassel  Ljubljana  \
True                                                                           
Basel          27         2     164         110       351       8       2297   
Belgrade        0         0       1          19        16       0        961   
Budapest        0         0       0           0        12       0        145   
Debilt          0         0       2           0        12       0         51   
Dusseldorf      0         0       0           2         0       0         13   
Heathrow        0         0       5           1        17       0         27   
Kassel          0         0       1           0         0       0          6   
Ljubljana       0         0       1           0         3       0         49   
Maastricht      0         0       1           0         1       0          4   
Madrid          0         0      20           8        55       0        171 

####  The model only recognized 11 cities. The accuracy score for the given confusion matrix is approximately 18.6%

## CNN Model 2

In [28]:
# Adjust model hyperparameters
epochs = 128
batch_size = 32
n_hidden = 64
kernel_size = 4 # Adjust the kernel size here

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

# Implement complex layers
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=kernel_size, activation='relu', input_shape=(timesteps, input_dim)))  # Use the adjusted kernel size
model.add(Dense(64, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #relu #sigmoid #tanh #softmax

# Build model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Run model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/128


C:\Users\Quinn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1535 - loss: 1526.6324
Epoch 2/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1202 - loss: 36258.8906
Epoch 3/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1151 - loss: 179701.3594
Epoch 4/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1142 - loss: 490314.3750
Epoch 5/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1066 - loss: 803812.2500
Epoch 6/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1159 - loss: 1101852.5000
Epoch 7/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1068 - loss: 1340660.0000
Epoch 8/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1131 - loss: 1876364.7500
Epoch 9/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1260 - loss: 2063242.0000
Epoch 10/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1206 - loss: 2375708.0000
Epoch 11/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1228 - loss: 2921123.

503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1235 - loss: 256623456.0000
Epoch 65/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1264 - loss: 255635392.0000
Epoch 66/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1314 - loss: 298132096.0000
Epoch 67/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1243 - loss: 292167136.0000
Epoch 68/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1202 - loss: 293208096.0000
Epoch 69/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1228 - loss: 302259296.0000
Epoch 70/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1267 - loss: 320233952.0000
Epoch 71/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1282 - loss: 330478400.0000
Epoch 72/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1302 - loss: 364046784.0000
Epoch 73/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1288 - loss: 384163488.0000
Epoch 74/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1305 - loss: 2165175808.0000
Epoch 127/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1220 - loss: 2348584192.0000
Epoch 128/128
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1254 - loss: 2398346752.0000


#### Notes: Loss decrease! 

In [29]:
# Define confusion matrix to see results.

def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([cities[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([cities[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [30]:
# Print results.

print(confusion_matrix(y_test, model.predict(X_test)))

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        Basel  Belgrade  Budapest  Debilt  Dusseldorf  Ljubljana  Madrid  \
True                                                                           
Basel           3       101        28       2        1384          7    2692   
Belgrade        0        45         0       0         256          0     992   
Budapest        0         4         0       0          67          0     169   
Debilt          0         0         0       0          36          0      60   
Dusseldorf      0         0         0       0          11          0      19   
Heathrow        0         0         0       0          38          0      59   
Kassel          0         0         0       0           7          0       5   
Ljubljana       0         0         0       0          15          0      56   
Maastricht      0         0         0       0           2          0       7   
Madrid          0         2         1       0         113          0     421   

#### The model finally recognized all cities. The accuracy also increased, approximately 38.1%

## CNN Model 3

In [31]:
# Adjust model hyperparameters
epochs = 256
batch_size = 32
n_hidden = 64
kernel_size = 4 # Adjust the kernel size here

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

# Implement complex layers
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=kernel_size, activation='relu', input_shape=(timesteps, input_dim)))  # Use the adjusted kernel size
model.add(Dense(64, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #relu #sigmoid #tanh #softmax

# Build model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Run model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/256


C:\Users\Quinn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


503/503 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1244 - loss: 1168.4932
Epoch 2/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1124 - loss: 26319.2734
Epoch 3/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1121 - loss: 78678.6641
Epoch 4/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1083 - loss: 151171.1250
Epoch 5/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1102 - loss: 206676.8594
Epoch 6/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1119 - loss: 250229.7969
Epoch 7/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1248 - loss: 299346.8750
Epoch 8/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1203 - loss: 391778.8125
Epoch 9/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1197 - loss: 551998.4375
Epoch 10/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1145 - loss: 619002.6875
Epoch 11/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1149 - loss: 963969.3125
Ep

503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1213 - loss: 222725392.0000
Epoch 65/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1227 - loss: 265525632.0000
Epoch 66/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1238 - loss: 266087888.0000
Epoch 67/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1209 - loss: 281523168.0000
Epoch 68/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1186 - loss: 294150464.0000
Epoch 69/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1266 - loss: 305022688.0000
Epoch 70/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1215 - loss: 334932128.0000
Epoch 71/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1260 - loss: 339856960.0000
Epoch 72/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1322 - loss: 359923680.0000
Epoch 73/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1208 - loss: 383862944.0000
Epoch 74/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1197 - loss: 2072110592.0000
Epoch 127/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1227 - loss: 2150673152.0000
Epoch 128/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1251 - loss: 2220798464.0000
Epoch 129/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1189 - loss: 2395525376.0000
Epoch 130/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1240 - loss: 2458628096.0000
Epoch 131/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1178 - loss: 2491227136.0000
Epoch 132/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1233 - loss: 2367490816.0000
Epoch 133/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1220 - loss: 2651887616.0000
Epoch 134/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1190 - loss: 2503233024.0000
Epoch 135/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1289 - loss: 2743630848.0000
Epoch 136/256
503/503 ━━━━━━━━━━━━

503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1255 - loss: 7748709888.0000
Epoch 189/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1235 - loss: 8144630784.0000
Epoch 190/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1236 - loss: 8046223360.0000
Epoch 191/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1342 - loss: 8324023296.0000
Epoch 192/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1283 - loss: 8596877312.0000
Epoch 193/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1170 - loss: 8465705472.0000
Epoch 194/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1205 - loss: 8507407360.0000
Epoch 195/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1242 - loss: 8920349696.0000
Epoch 196/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1206 - loss: 9174248448.0000
Epoch 197/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1186 - loss: 10111646720.0000
Epoch 198/256
503/503 ━━━━━━━━━━━

503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1279 - loss: 20051103744.0000
Epoch 250/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1234 - loss: 20013639680.0000
Epoch 251/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1283 - loss: 21016946688.0000
Epoch 252/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1222 - loss: 19312881664.0000
Epoch 253/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1198 - loss: 21317765120.0000
Epoch 254/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1169 - loss: 20631717888.0000
Epoch 255/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1186 - loss: 22411769856.0000
Epoch 256/256
503/503 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1242 - loss: 22421733376.0000


In [32]:
# Define confusion matrix to see results.

def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([cities[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([cities[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [33]:
# Print results.

print(confusion_matrix(y_test, model.predict(X_test)))

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        Basel  Belgrade  Budapest  Debilt  Dusseldorf  Heathrow  Kassel  \
True                                                                          
Basel           9       645        16      17         157       105      89   
Belgrade        0       377         5       0           7         2       1   
Budapest        0        57         0       0           2         0       0   
Debilt          0         2         0       0           1         1       0   
Dusseldorf      0         2         0       0           0         0       0   
Heathrow        0         2         0       0           1         5       0   
Kassel          0         1         0       0           0         1       0   
Ljubljana       0        26         2       0           0         1       0   
Maastricht      0         3         0       0           0         0       0   
Madrid          1       114         2       0           5        24       0   
Munchen    

#### The loss decreased and the model recognized all the cities, but the accuracy also decreased, down to approximately 9.7%. Let's try a final one

## CNN Model 3

In [34]:
# Adjust model hyperparameters
epochs = 256
batch_size = 64
n_hidden = 128
kernel_size = 4 # Adjust the kernel size here

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

# Implement complex layers
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=kernel_size, activation='relu', input_shape=(timesteps, input_dim)))  # Use the adjusted kernel size
model.add(Dense(64, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #relu #sigmoid #tanh #softmax

# Build model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Run model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/256


C:\Users\Quinn\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


252/252 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0787 - loss: 316.2588
Epoch 2/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1134 - loss: 17377.1348
Epoch 3/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1122 - loss: 91802.6719
Epoch 4/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1269 - loss: 205862.6875
Epoch 5/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1151 - loss: 355587.6875
Epoch 6/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1033 - loss: 501925.8125
Epoch 7/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1174 - loss: 525132.5000
Epoch 8/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1208 - loss: 557035.6250
Epoch 9/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1126 - loss: 685845.0000
Epoch 10/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1109 - loss: 435223.2188
Epoch 11/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1088 - loss: 382129.9375
Epo

252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1186 - loss: 8476215.0000
Epoch 66/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1206 - loss: 7379777.5000
Epoch 67/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1336 - loss: 7519591.5000
Epoch 68/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1223 - loss: 4633150.5000
Epoch 69/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1152 - loss: 6081266.5000
Epoch 70/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1168 - loss: 4330738.5000
Epoch 71/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1212 - loss: 5485474.5000
Epoch 72/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1276 - loss: 3906756.0000
Epoch 73/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1193 - loss: 10140624.0000
Epoch 74/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1265 - loss: 12542612.0000
Epoch 75/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.113

252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1282 - loss: 18558990.0000
Epoch 129/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1098 - loss: 25220854.0000
Epoch 130/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1229 - loss: 15126591.0000
Epoch 131/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1036 - loss: 20550390.0000
Epoch 132/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1312 - loss: 17337380.0000
Epoch 133/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1199 - loss: 15764491.0000
Epoch 134/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1209 - loss: 14706449.0000
Epoch 135/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1268 - loss: 14112972.0000
Epoch 136/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1199 - loss: 14385690.0000
Epoch 137/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1253 - loss: 15702955.0000
Epoch 138/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1257 - loss: 34798932.0000
Epoch 192/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1272 - loss: 26319030.0000
Epoch 193/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1271 - loss: 28900824.0000
Epoch 194/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1261 - loss: 29793740.0000
Epoch 195/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1114 - loss: 42765336.0000
Epoch 196/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1344 - loss: 27399396.0000
Epoch 197/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1196 - loss: 33177472.0000
Epoch 198/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1291 - loss: 28066706.0000
Epoch 199/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1328 - loss: 36233856.0000
Epoch 200/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1182 - loss: 26856402.0000
Epoch 201/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1301 - loss: 160389984.0000
Epoch 254/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1139 - loss: 162208416.0000
Epoch 255/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1169 - loss: 165853120.0000
Epoch 256/256
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1207 - loss: 136276784.0000


In [35]:
# Define confusion matrix to see results.

def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([cities[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([cities[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [36]:
# Print results.

print(confusion_matrix(y_test, model.predict(X_test)))

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        Basel  Belgrade  Budapest  Debilt  Dusseldorf  Heathrow  Kassel  \
True                                                                          
Basel           7       246         1     127         176         9      26   
Belgrade        0       158         0       0           1         0       0   
Budapest        0        22         0       0           0         0       0   
Debilt          0         0         0       0           0         0       0   
Dusseldorf      0         0         0       0           1         0       0   
Heathrow        0         0         0       0           3         0       0   
Kassel          0         1         0       0           0         0       0   
Ljubljana       0         4         0       0           0         0       0   
Maastricht      0         0         0       0           0         0       0   
Madrid          0        29         0       5          10         0       0   
Munchen    

#### Eventhough the loss continued to decrease but the accuracy didn't improve much, approximately 29.7%. And the model could not recognized 1 city.

In [37]:
# Export the clean dataframe
climate.to_csv(os.path.join(path, '02 Data Sets', 'cleaned_climate.csv'))